In [ ]:
import shap

# 1. Interpretabilidad con SHAP
explainer = shap.TreeExplainer(best_model.named_steps['model'])
shap_values = explainer.shap_values(X_test_processed)

shap.summary_plot(shap_values, X_test_processed, feature_names=feature_names)

# 2. Curva ROC Multiclase
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

y_test_bin = label_binarize(y_test, classes=best_model.classes_)
y_pred_proba = best_model.predict_proba(X_test)

plt.figure(figsize=(10,8))
for i in range(len(best_model.classes_)):
    fpr, tpr, _ = roc_curve(y_test_bin[:,i], y_pred_proba[:,i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{best_model.classes_[i]} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.title('Curva ROC Multiclase')
plt.legend(loc='lower right')
plt.show()